# Project) 웹 스크래핑을 이용하여 나만의 비서를 만드시오.
- [조건]
    1. 네이버에서 오늘 서울의 날씨정보를 가져온다.
    2. 헤드라인 뉴스 3건을 가져온다.
    3. IT 뉴스 3건을 가져온다.
    4. 해커스 어학원 홈페이지에서 오늘의 영어 회화 지문을 가져온다.
- [출력 예시]<br>
    [오늘의 날씨]<br>
    흐림, 어제보다 00° 높아요<br>
    현재 00°C (최저 00°C / 최고 00°C)<br>
    오전 강수확률 00% / 오후 강수확률 00%<br>
    <br>
    미세먼지 00㎍/㎥ 좋음<br>
    초미세먼지 00㎍/㎥ 좋음<br>
- [출력 예시]<br>
    [헤드라인 뉴스]
    1. 무슨 무슨 일이,,,
    (링크 : https://...)
    2. 어떤 어떤 일이,,,
    (링크 : https://...)
    3. ? ? 일이,,,
    (링크 : https://...)

In [102]:
# [오늘의 날씨]
# 흐림, 어제보다 00° 높아요
# 현재 00°C (최저 00°C / 최고 00°C)
# 오전 강수확률 00% / 오후 강수확률 00%

# 미세먼지 00㎍/㎥ 좋음
# 초미세먼지 00㎍/㎥ 좋음
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%84%9C%EC%9A%B8+%EB%82%A0%EC%94%A8"

res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")

weather_info = soup.select_one('div.weather_info')
weather_info.select_one('div.temperature_text strong').span.decompose() # span "현재 온도" 제거
current_celsius = weather_info.select_one('div.temperature_text strong').get_text()
temperature_info = weather_info.select_one('div.temperature_info > p').get_text().strip().split("  ")

today_info = soup.select_one('div.list_box > ul.week_list > li.week_item.today')

# 최저, 최고 기온 구하기
highest_temperature = today_info.select_one('span.highest').get_text()
lowest_temperature = today_info.select_one('span.lowest').get_text()
highest_temperature = re.sub(r"[가-힣]", "", highest_temperature)
lowest_temperature = re.sub(r"[가-힣]", "", lowest_temperature)

# 오전, 오후 강수량 구하기
precipitation = today_info.select('div.cell_weather span.weather_inner')
m_precipitation = precipitation[0].select_one('span.rainfall').get_text()
a_precipitation = precipitation[1].select_one('span.rainfall').get_text()

# 미세먼지 구하기 => 셀레니움 사용
# headless 옵션 적용
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080")

browser = webdriver.Chrome(executable_path="./chromedriver", options=options)
browser.get(url)

browser.find_element_by_xpath("//*[@id='main_pack']/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[2]/div/div[3]/ul/li[1]/a").click()
dust = browser.find_elements_by_class_name('grade')
fine_dust = dust[0].text.split(' ')
u_fine_dust = dust[1].text.split(' ')

browser.quit()

print("[오늘의 날씨]")
print(f"{temperature_info[1]}, {temperature_info[0]}")
print(f"현재 {current_celsius}C (최저 {lowest_temperature}C / 최고 {highest_temperature}C)")
print(f"오전 강수확률 {m_precipitation} / 오후 강수확률 {a_precipitation}")
print(f"미세먼지 {fine_dust[0]}㎍/㎥ {fine_dust[1]}")
print(f"초미세먼지 {u_fine_dust[0]}㎍/㎥ {u_fine_dust[1]}")

<ipython-input-102-0e1a7b706d96>:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="./chromedriver", options=options)
<ipython-input-102-0e1a7b706d96>:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath("//*[@id='main_pack']/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[2]/div/div[3]/ul/li[1]/a").click()


[오늘의 날씨]
흐림, 어제보다 5° 높아요
현재 4°C (최저 0°C / 최고 6°C)
오전 강수확률 60% / 오후 강수확률 20%
미세먼지 71㎍/㎥ 보통
초미세먼지 52㎍/㎥ 나쁨


<ipython-input-102-0e1a7b706d96>:46: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  dust = browser.find_elements_by_class_name('grade')


- decompose()
    - select_one 으로 찾은 태그 내부의 자식 태그들 중 해당 태그를 처음부터 하나씩 제거
    - ex) weather_info.select_one('div.temperature_text strong').span.decompose()

In [108]:
# 1. 무슨 무슨 일이,,, (링크 : https://...)
# 2. 어떤 어떤 일이,,, (링크 : https://...)
# 3. ? ? 일이,,, (링크 : https://...)

import requests
from bs4 import BeautifulSoup

url = "https://news.naver.com/"

journal_infos = list()

res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")

journals = soup.select('div.cjs_journal_wrap._item_contents')[:3]

for journal in journals:
    journal_title = journal.select_one('a.cjs_news_a div.cjs_t').get_text()
    journal_link = journal.select_one('a.cjs_news_a')['href']
    journal_infos.append([journal_title, journal_link])

for index, journal_info in enumerate(journal_infos):
    print(f"{index+1}. {journal_info[0]} (링크 : {journal_info[1]})")

1. 코로나보다 기후문제가 더 시급하다 (링크 : https://n.news.naver.com/article/033/0000043515)
2. “삼성 이러다 큰일 나겠네” 폴더블폰보다 10배 더 팔린 ‘이것’ (링크 : https://n.news.naver.com/article/016/0001930873)
3. 올해의 여성노동운동상에 'LG트윈타워 청소노동자' (링크 : https://n.news.naver.com/article/310/0000092936)


In [16]:
import requests
from bs4 import BeautifulSoup

url = "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105"

it_journal_infos = list()

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.content, "html.parser")

it_journals = soup.select("div._persist div.cluster_group._cluster_content")[:3]

for journal in it_journals:
    it_journal = journal.select_one("div.cluster_body ul.cluster_list li.cluster_item div.cluster_text > a")
    title = it_journal.get_text()
    link = it_journal['href']
    it_journal_infos.append([title, link])

for index, info in enumerate(it_journal_infos):
    print(f"{index+1}. {info[0]} (링크 : {info[1]})")

1. "애플 추월하겠다" 호언장담…샤오미가 꺼내든 비장의 무기 (링크 : https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=015&aid=0004646580)
2. KT 사고나면 SKT 네트워크망 쓴다...과기정통부  '타사망 긴급 연동 방안' 마련 (링크 : https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=293&aid=0000037454)
3. 2015년 스페이스엑스 폭발도 헬륨탱크 때문…“누리호 실패도 개발 과정” (링크 : https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=028&aid=0002573480)


## ERROR
- ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
    - 응답 HTTP가 에러 코드로 반환된 상황
    - user agent 값을 변경하고 실행하니 정상적으로 동작함.

In [50]:
import requests
from bs4 import BeautifulSoup

url = "https://www.hackers.co.kr/?c=s_eng/eng_contents/I_others_english&keywd=haceng_submain_lnb_eng_I_others_english&logger_kw=haceng_submain_lnb_eng_I_others_english"

res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.content, "html.parser")

fingerprints = soup.select('div.conv_main div.conv_container div.conv_txtBox')

k_fingerprint = fingerprints[0].select_one('div.conv_txt')
e_fingerprint = fingerprints[1].select_one('div.conv_txt')

# 영어 지문 텍스트 스크래핑
today_e_expression = e_fingerprint.select_one('b').get_text()
e_sentences = e_fingerprint.select('span.conv_sub b')
e_sentence = list()
for s in e_sentences:
    e_sentence.append(s.get_text().strip())

# 한글 지문 텍스트 스크래핑
today_k_expression = k_fingerprint.select_one('b').get_text()
k_sentences = k_fingerprint.select('span.conv_sub b')
k_sentence = list()
for s in k_sentences:
    k_sentence.append(s.get_text().strip())

e_sentence.insert(0, today_e_expression)
k_sentence.insert(0, today_k_expression)

print("(영어 지문)")
print(f"Today's Expression : {e_sentence[0]}\n")
for s in e_sentence[1:]:
    print(s + "\n")

print("(한글 지문)")
print(f"오늘의 표현 : {k_sentence[0]}\n")
for s in k_sentence[1:]:
    print(s + "\n")

(영어 지문)
Today's Expression : This project could be very beneficial to our firm.

Mr. Kim: I would like to suggest that this project could be very beneficial to our firm.

Mrs. Park: But you recently indicated that it would have to be renegotiated to even reach a profit.

Mr. Kim: Yes, but if you take into account the sales projections in my proposal you will see that significant profits can be realized.

Mrs. Park: Yes, that does appear to be the case.

(한글 지문)
오늘의 표현 :  이번 프로젝트는 우리 회사에 많은 이익을 가져다 줄 수도 있을 거예요.

Mr. Kim: 이번 프로젝트는 우리 회사에 많은 이익을 가져다 줄 수도 있을 거예요.

Mrs. Park: 하지만 당신은 최근에 이익을 얻으려면 재협상 할 필요가 있다고 하셨잖아요.

Mr. Kim: 네, 하지만 제가 보고 드린 판매 예상을 고려하신다면 분명 이익이 보장될 것입니다.

Mrs. Park: 네, 보고서를 보면 그렇게 생각이 됩니다.

